# Sample Crash/etc Data Set - Received Dec. 7th 2015

All columns names and allowed values can be referenced in the __CrashDataDictionary.pdf__ file. 

# Crash Data

### Load Data

In [1]:
#Load Packages
import pandas as pd
import numpy as np
from IPython.display import display, HTML
low_memory = False

local_path = "/Users/michaeldowd/"
#Load the csv's 
clion_lookup = pd.read_csv(local_path + "Google Drive/Dowd_Local/Data/1st_Sample/c_lion_node_lookup_sample.csv")
crashes =      pd.read_csv(local_path + "Google Drive/Dowd_Local/Data/1st_Sample/crashes_sample.csv")
vehicles =     pd.read_csv(local_path + "Google Drive/Dowd_Local/Data/1st_Sample/vehicle_sample.csv")
factors =      pd.read_csv(local_path + "Google Drive/Dowd_Local/Data/1st_Sample/factor_sample.csv")

print "done"

/Users/michaeldowd/Library/Enthought/Canopy_64bit/User/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2902: DtypeWarning: Columns (5,32,73) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/michaeldowd/Library/Enthought/Canopy_64bit/User/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2902: DtypeWarning: Columns (4,19) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


done


### Crashes Column List

In [2]:
display(HTML(pd.DataFrame(list(crashes.columns)).to_html()))


,0
0,gid
1,case_num
2,case_yr
3,ref_mrkr
4,accd_dte
5,road_sys
6,num_of_fat
7,num_of_inj
8,reportable
9,police_dep


### Null/NaN Totals by Column

See total & percent of null values by each column (where null values are present)

In [3]:
#How many nulls in each column (only show columns with more than zero nulls)
crash_nulls = crashes.isnull().sum()
crash_nulls = crash_nulls[crash_nulls > 0]

def getPercNull(df, master_df):
    #Function for calculation percent of records that are null
    percents = []
    for b in df.iterrows():
        try:
            percents.append(b[1].Count/float(len(master_df[b[0]])))
        except:
            percents.append("NA")
    
    df['perc_null'] = percents

crash_nulls = pd.DataFrame(crash_nulls, columns = ["Count"])
getPercNull(crash_nulls, crashes)

display(HTML(crash_nulls.to_html()))
print "Total Records : " , len(crashes)

,Count,perc_null
ref_mrkr,31005,0.979776
police_dep,7888,0.249265
intersect_,31366,0.991183
precinct,7909,0.249929
locn,2,0.000063
ext_of_inj,8092,0.255712
accd_tme,1210,0.038237
rpt_agcy,3360,0.106178
traf_way,2114,0.066804
rdway_acc_,2114,0.066804


Total Records :  31645


### Determine Priority Columns
#### Priority columns are those columns that may be used as an explanatory variable

In [4]:
priority_cols_crashes = ['case_yr', "road_sys", "reportable", "accd_typ", "num_of_veh", "traf_cntl", "light_cond", \
    "weather", "road_char","road_surf_", "collision_","ped_loc",  \
                 "ped_actn", "ext_of_inj","regn_cnty_", "dmv_accd_c", "err_cde", \
                "highway_in", "intersect1" ]

In [39]:
def summarize(column_list, master_df, include_sums = True):
    """
    Cycles through the columns in the priority cols list and does a simple aggregate, 
    total number of cases for each column, the sum of fatalities, and the sum of injuries.
    """
    print 5*">", "START", 5 * "<"
    for col in column_list:
        print ">"*20
        print col.upper()
        
        if include_sums:
            out =  master_df[[col, 'crashid', "num_of_fat", "num_of_inj"]].groupby(col)
            aggout =  out.agg({'crashid':{'count' :  'count' },
                          'num_of_fat' : {'sum' : 'sum'}, 
                          'num_of_inj' : {'sum' : 'sum'}
                          })
        elif include_sums == "Vehicles":
            out =  master_df[[col, 'crashid', "num_of_fat", "num_of_inj"]].groupby(col)
            aggout =  out.agg({'crashid':{'count' :  'count' },
                                'num_of_fat' : {'sum' : lambda x: np.sum(x)}
              })
        else:
            out =  master_df[[col, 'crashid']].groupby(col)
            aggout =  out.agg({'crashid':{'count' :  'count' }
              })
        display(HTML(aggout.to_html()))
    print 5*">", "END", 5 * "<"
    return aggout


### Crash Data Summary of Priority Columns 

Total number of indcidents, fatalities and injuries by each priority column.


In [6]:
print 5*">", "Crash Data", 5 * "<"
summarize(priority_cols_crashes, crashes)

>>>>> Crash Data <<<<<
>>>>> START <<<<<
>>>>>>>>>>>>>>>>>>>>
CASE_YR


,num_of_inj,crashid,num_of_fat
,sum,count,sum
case_yr,,,
2001,3852,3322,9
2002,3520,2730,8
2003,3019,2351,5
2004,2531,2087,8
2005,2300,1927,4
2006,2241,2089,4
2007,2142,2304,7
2008,2158,2313,9


>>>>>>>>>>>>>>>>>>>>
ROAD_SYS


,num_of_inj,crashid,num_of_fat
,sum,count,sum
road_sys,,,
9,6,7,0
12,7061,7062,18
01,87,60,0
02,8,6,0
03,63,49,0
04,17865,15150,37
05,20,16,0
09,44,46,0


>>>>>>>>>>>>>>>>>>>>
REPORTABLE


...

>>>>>>>>>>>>>>>>>>>>
HIGHWAY_IN


,num_of_inj,crashid,num_of_fat
,sum,count,sum
highway_in,,,
N,33982,31626,86
Y,6,19,0


>>>>>>>>>>>>>>>>>>>>
INTERSECT1


,num_of_inj,crashid,num_of_fat
,sum,count,sum
intersect1,,,
N,2715,2874,9
Y,26211,24023,66


>>>>> END <<<<<


### Data Cleaning - INCOMPLETE - may be needed for other columns if we use Pandas for loading

In [15]:
#without using column value type enforcement - some mixed type colums emerged, code below corrected those in roadsys
crashes.road_sys.loc[crashes[crashes.road_sys == 12].index] = '12'
crashes.road_sys.loc[crashes[crashes.road_sys == 9].index] = '9'

pd.unique(crashes['road_sys'].ravel())

array(['04', '12', '??', '03', 'XX', '11', '09', '02', '05', '01', '15',
       '10', '9'], dtype=object)

## Vehicles Data


Vehicles Data Column List

In [8]:
pd.DataFrame(vehicles.columns, columns = ["Cols"])

,Cols
0,case_num
1,case_yr
2,veh_seq_num
3,rgst_typ
4,body_typ
5,veh_typ
6,pre_accd_actn
7,second_event
8,veh_dirn_of_trav
9,haz_cargo_ind


### Null/NaN Totals by Column

In [9]:
#How many nulls in each column (only show columns with more than zero nulls)
number_of_nulls = vehicles.isnull().sum()
number_of_nulls = number_of_nulls[number_of_nulls > 0]

nulls_df = pd.DataFrame(number_of_nulls, columns = ["Count"])
getPercNull(nulls_df, vehicles)

display(HTML(nulls_df.to_html()))
        
print "Total Records : " , len(vehicles)

,Count,perc_null
age,11723,0.176626
sex,14567,0.219475
occupant_num,1221,0.018396
rgst_wgt,31086,0.468360
cit_ind,14349,0.216191
drvr_lic_st,21012,0.316579
veh_lic_st,14218,0.214217
hazmat_plac_ind,41529,0.625701
tow_ind,4419,0.066579


Total Records :  66372


### Determine Priority Columns

In [10]:
priority_cols_vehicles = [
    'case_yr', 'rgst_typ', 'body_typ', 'veh_typ','pre_accd_actn', 'age', 'sex', 'rgst_wgt'
]

In [11]:
#Join Crashes(just injuries and Fatalities) & Vehicles
crash_short = crashes[['case_num','crashid', 'num_of_fat', 'num_of_inj']]
merge_Vehicles = pd.merge(vehicles, crash_short, on='crashid')
grouped_by_Case_merge_vehicles = merge_Vehicles.groupby('crashid')

In [12]:
test = merge_Vehicles[['crashid','num_of_fat','num_of_inj']].groupby('crashid').sum()
# test.head()
merge_Vehicles.head()

,case_num_x,case_yr,veh_seq_num,rgst_typ,body_typ,veh_typ,pre_accd_actn,second_event,veh_dirn_of_trav,haz_cargo_ind,...,cit_ind,drvr_lic_st,veh_lic_st,hazmat_plac_ind,tow_ind,crashid,tck_bus_clsf,case_num_y,num_of_fat,num_of_inj
0,1103203,2001,1,ZZ,32,0,01,ZZ,3,N,...,NaN,NaN,NaN,NaN,NaN,11032032001,ZZ,1103203,0,1
1,1103203,2001,2,ZZ,36,6,ZZ,ZZ,Z,N,...,NaN,NaN,NaN,NaN,NaN,11032032001,ZZ,1103203,0,1
2,1103232,2001,1,16,13,2,01,ZZ,1,N,...,NaN,NaN,NaN,NaN,NaN,11032322001,ZZ,1103232,0,0
3,1103232,2001,2,16,68,2,03,ZZ,5,N,...,NaN,NaN,NaN,NaN,NaN,11032322001,ZZ,1103232,0,0
4,1103241,2001,3,54,13,2,07,ZZ,1,N,...,NaN,NaN,NaN,NaN,NaN,11032412001,ZZ,1103241,0,1


### Vehicles Dataset Summary
Note: There is double counting present in the sum results in tables below, as each row in the vehicles table represents a party in the accident (vehicles, ped or bike). Data is just to provide an idea of the distribution of value for different fields. Injuries/Fatalities are not mapped to specific people/vehicles. I.E. If a crash occured and we know there was a bike and a car and we know the age/sex/etc of the two people involved (driver / rider) we **do not** know which person was injured/killed. 

In [13]:
summarize(priority_cols_vehicles, merge_Vehicles, include_sums=True)

>>>>> START <<<<<
>>>>>>>>>>>>>>>>>>>>
CASE_YR


,num_of_inj,crashid,num_of_fat
,sum,count,sum
case_yr,,,
2001,8304,6943,18
2002,7715,5744,14
2003,6569,4951,11
2004,5507,4429,20
2005,5103,4068,9
2006,4969,4410,14
2007,4720,4839,30
2008,4609,4807,18


>>>>>>>>>>>>>>>>>>>>
RGST_TYP


,num_of_inj,crashid,num_of_fat
,sum,count,sum
rgst_typ,,,
11,592,577,1
12,16,24,0
15,1,1,0
16,34899,30082,51
19,174,119,0
20,1,1,0
21,2,2,0
26,25,21,0


>>>>>>>>>>>>>>>>>>>>
BODY_TYP


...

>>>>>>>>>>>>>>>>>>>>
SEX


,num_of_inj,crashid,num_of_fat
,sum,count,sum
sex,,,
F,17192,14472,35
M,44162,37077,126
U,28,29,0
f,24,41,0
m,138,185,0
u,1,1,0


>>>>>>>>>>>>>>>>>>>>
RGST_WGT


,num_of_inj,crashid,num_of_fat
,sum,count,sum
rgst_wgt,,,
0,1,1,0
1,1,2,0
4,35,26,0
5,342,264,0
6,1,2,0
7,12,12,0
8,16,4,0
9,1,1,0


>>>>> END <<<<<


## Factor Dataset

In [14]:
print factors.head()
len(factors)

   case_num  case_yr  veh_seq_num  aprnt_seq_num aprnt_fctr       crashid
0  31708092     2005            1              1         ZZ  317080922005
1  31722193     2005            2              2         ZZ  317221932005
2  31720835     2005            1              1         42  317208352005
3  31720835     2005            3              2         ZZ  317208352005
4  31725298     2005            1              2         YY  317252982005


132744

In [27]:
vehicles['uid'] = vehicles.crashid.map(str) + "_" + vehicles.veh_seq_num.map(str)
factors['uid'] = factors.crashid.map(str) + "_" + factors.veh_seq_num.map(str)

vehicles.crashid

0         11032032001
1         11032032001
2         11032322001
3         11032322001
4         11032412001
5         11032412001
6         11032412001
7         11033772001
8         11033772001
9         11033772001
10        11033772001
11        11046112001
12        11046112001
13        11047672001
14        11047672001
15        11047922001
16        11047922001
17        11048012001
18        11048012001
19        11048012001
20        11059052001
21        11059052001
22        11059432001
23        11059432001
24        11059502001
25        11059502001
26        11060692001
27        11060692001
28        11060832001
29        11060832001
             ...     
66342    356345442014
66343    356345442014
66344    356345472014
66345    356345472014
66346    356372572014
66347    356372572014
66348    356430272014
66349    356430272014
66350    356479652014
66351    356479652014
66352    356479652014
66353    356479662014
66354    356479662014
66355    356479662014
66356    3

In [40]:
factor_out = summarize(["aprnt_fctr"], factors, include_sums = False)
factor_out.sort('')

>>>>> START <<<<<
>>>>>>>>>>>>>>>>>>>>
APRNT_FCTR


,crashid
,count
aprnt_fctr,
01,4
02,353
03,579
04,6192
05,719
06,29
07,2634
08,99


>>>>> END <<<<<


pandas.core.frame.DataFrame

In [62]:
print factor_out.columns.get_level_values(1)
test = factor_out.iloc[:, factor_out.columns.get_level_values(1)=='count']

Index([u'count'], dtype='object')


In [78]:
pd.DataFrame(test['crashid']['count']).sort_values('count')

,count
aprnt_fctr,
51,1
31,2
43,2
34,2
33,4
32,4
01,4
49,5
67,6
